In [1]:
import os, json, re, requests, base64, time, datetime

import psycopg2
import sqlalchemy as sa
from urllib.parse import urlparse, parse_qs

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [2]:
db_creds_file = r"D:\personal\creds\psql\db_creds.json"
with open(db_creds_file) as f:
    db_creds = json.load(f)

uid = db_creds["uid"]
pw = db_creds["password"]
host = "localhost"
port = 5432
db = "spotify"

engine = sa.create_engine(f"postgresql+psycopg2://{uid}:{pw}@{host}:{port}/{db}")
conn = engine.connect()

In [3]:
stmt = """
select id
from audio.track_info
"""

df = pd.read_sql(stmt, conn)

In [4]:
genres = r"D:\data_projects\spotify\dump"

In [5]:
df["genres"] = np.empty((len(df), 0)).tolist()

for genre in os.listdir(genres):
    for file in os.listdir(os.path.join(genres, genre)):
        track = file.split(".")[0]
        df.loc[df[df["id"]==track].index, "genres"].apply(lambda x: x.append(genre))

In [7]:
df.head()

,id,genres
0,69uJi5QsBtqlYkGURTBli8,"[acoustic, sad]"
1,7ilWEHob3AYG3lAJlKEpDR,[acoustic]
2,0SuG9kyzGRpDqrCWtgD6Lq,"[acoustic, british, chill, guitar, happy, rainy-day, road-trip, sad, singer-songwriter, sleep, study, summer]"
3,3K9PY33TKt25UXX6YymFQq,"[acoustic, chill, folk, rainy-day, singer-songwriter, sleep]"
4,7paYa83IYmg2dvRu3NvcKL,[acoustic]


In [8]:
stmt = """
DROP TABLE IF EXISTS audio.track_genres;

CREATE TABLE audio.track_genres (
id                            text PRIMARY KEY,
genres                        text[]
);
"""

[conn.execute(s) for s in stmt.split(";") if s != "\n"]

df.to_sql(name="track_genres", con=conn, schema="audio", index=False, if_exists="append")